In [1]:
import tensorflow as tf
import numpy as np
import os, re
from tensorflow.python.layers.core import Dense
import datetime

# for bleu score
# !pip install nltk
from nltk.translate import bleu_score
from nltk.translate.bleu_score import SmoothingFunction

D:\miniconda3\envs\seq2seq_tf1\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\miniconda3\envs\seq2seq_tf1\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\miniconda3\envs\seq2seq_tf1\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\miniconda3\envs\seq2seq_tf1\lib\site-packages\tensorflow\python\framework\dtypes.py:5

In [2]:
print(tf.__version__)

1.14.0


In [3]:
MAX_CHAR_PER_LINE = 20

def load_sentences(path):
    with open(path, 'r', encoding="ISO-8859-1") as f:
        data_raw = f.read().encode('ascii', 'ignore').decode('UTF-8').lower()
        data_alpha = re.sub('[^a-z\n]+', ' ', data_raw)
        data = []
        for line in data_alpha.split('\n'):
            data.append(line[:MAX_CHAR_PER_LINE])
    return data

In [4]:
def extract_character_vocab(data):
    special_symbols = ['<PAD>', '<UNK>', '<GO>',  '<EOS>']
    set_symbols = set([character for line in data for character in line])
    all_symbols = special_symbols + list(set_symbols)
    int_to_symbol = {word_i: word for word_i, word in enumerate(all_symbols)}
    symbol_to_int = {word: word_i for word_i, word in int_to_symbol.items()}
    return int_to_symbol, symbol_to_int

input_sentences = load_sentences('data/words_input.txt')  
output_sentences = load_sentences('data/words_output.txt')  

input_int_to_symbol, input_symbol_to_int = extract_character_vocab(input_sentences)
output_int_to_symbol, output_symbol_to_int = extract_character_vocab(output_sentences)

In [5]:
input_int_to_symbol

{0: '<PAD>',
 1: '<UNK>',
 2: '<GO>',
 3: '<EOS>',
 4: 'o',
 5: 'v',
 6: 'm',
 7: 'd',
 8: 'l',
 9: 'u',
 10: 'a',
 11: 'h',
 12: 'j',
 13: 'k',
 14: 's',
 15: 'r',
 16: 'f',
 17: 'i',
 18: 'z',
 19: 'c',
 20: 't',
 21: 'q',
 22: 'y',
 23: 'g',
 24: 'b',
 25: 'p',
 26: 'x',
 27: ' ',
 28: 'n',
 29: 'e',
 30: 'w'}

In [6]:
output_int_to_symbol

{0: '<PAD>',
 1: '<UNK>',
 2: '<GO>',
 3: '<EOS>',
 4: 'o',
 5: 'v',
 6: 'm',
 7: 'd',
 8: 'l',
 9: 'u',
 10: 'a',
 11: 'h',
 12: 'j',
 13: 'k',
 14: 's',
 15: 'f',
 16: 'r',
 17: 'i',
 18: 'z',
 19: 'c',
 20: 't',
 21: 'q',
 22: 'y',
 23: 'g',
 24: 'b',
 25: 'p',
 26: 'x',
 27: ' ',
 28: 'n',
 29: 'e',
 30: 'w'}

In [7]:
# split the data 
print(len(input_sentences), len(output_sentences))

50000 50000


In [8]:
import random
 

# Shuffle two lists with same order
# Using zip() + * operator + shuffle()
temp = list(zip(input_sentences, output_sentences))
random.shuffle(temp)
res1, res2 = zip(*temp)
# res1 and res2 come out as tuples, and so must be converted to lists.
input_sentences, output_sentences = list(res1), list(res2)

# split the data
train_percent, evaluate_percent, test_percent = 0.7, 0.2, 0.1
length_sentences = len(input_sentences)

train_size = int(length_sentences*train_percent)

train_input_sentences = input_sentences[:train_size]

test_size = int(length_sentences*test_percent)

test_input_sentences = input_sentences[-test_size:]

evaluate_input_sentences = list(input_sentences[train_size:45000])



train_output_sentences = output_sentences[:train_size]
test_output_sentences = output_sentences[-test_size::]
evaluate_output_sentences = list(input_sentences[train_size:45000])


# for test ---- comment this part when finish the test --------------------------------
size = 10
train_input_sentences = train_input_sentences[:size]
evaluate_input_sentences = evaluate_input_sentences[:size]
test_input_sentences = test_input_sentences[:size]
train_output_sentences = train_output_sentences[:size]
evaluate_output_sentences = evaluate_output_sentences[:size]
test_output_sentences = test_output_sentences[:size]
# ------------------------------------------------------------------------------------

# Printing result
print(len(train_input_sentences), len(evaluate_input_sentences), len(test_input_sentences))
print(len(train_output_sentences), len(evaluate_output_sentences), len(test_output_sentences))

print(train_input_sentences[1], '\n', train_output_sentences[1])

10 10 10
10 10 10
you have known him l 
 since his beard was 


In [9]:
NUM_EPOCS = 300
RNN_STATE_DIM = 512
RNN_NUM_LAYERS = 2
ENCODER_EMBEDDING_DIM = DECODER_EMBEDDING_DIM = 64
 
# BATCH_SIZE = int(32)

# for test ----------------------------------------
BATCH_SIZE = int(5)
#--------------------------------------------------

LEARNING_RATE = 0.0003
 
INPUT_NUM_VOCAB = len(input_symbol_to_int)
OUTPUT_NUM_VOCAB = len(output_symbol_to_int)
print(OUTPUT_NUM_VOCAB)

31


In [10]:
# Encoder placeholders
encoder_input_seq = tf.placeholder(
    tf.int32, 
    [None, None],  
    name='encoder_input_seq'
)

encoder_seq_len = tf.placeholder(
    tf.int32, 
    (None,), 
    name='encoder_seq_len'
)
 
# Decoder placeholders
decoder_output_seq = tf.placeholder( 
    tf.int32, 
    [None, None],
    name='decoder_output_seq'
)

decoder_seq_len = tf.placeholder(
    tf.int32,
    (None,), 
    name='decoder_seq_len'
)

max_decoder_seq_len = tf.reduce_max( 
    decoder_seq_len, 
    name='max_decoder_seq_len'
)

In [11]:
def make_cell(state_dim):
    lstm_initializer = tf.random_uniform_initializer(-0.1, 0.1)
    return tf.contrib.rnn.LSTMCell(state_dim, initializer=lstm_initializer)
 
def make_multi_cell(state_dim, num_layers):
    cells = [make_cell(state_dim) for _ in range(num_layers)]
    return tf.contrib.rnn.MultiRNNCell(cells)

In [12]:
# Encoder embedding
 
encoder_input_embedded = tf.contrib.layers.embed_sequence(
    encoder_input_seq,     
    INPUT_NUM_VOCAB,        
    ENCODER_EMBEDDING_DIM  
)
 
 
# Encodering output
 
encoder_multi_cell = make_multi_cell(RNN_STATE_DIM, RNN_NUM_LAYERS)
 
encoder_output, encoder_state = tf.nn.dynamic_rnn(
    encoder_multi_cell, 
    encoder_input_embedded, 
    sequence_length=encoder_seq_len, 
    dtype=tf.float32
)
 
del(encoder_output)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where

In [13]:
decoder_raw_seq = decoder_output_seq[:, :-1]  
go_prefixes = tf.fill([BATCH_SIZE, 1], output_symbol_to_int['<GO>'])  
decoder_input_seq = tf.concat([go_prefixes, decoder_raw_seq], 1)  

In [14]:
decoder_embedding = tf.Variable(tf.random_uniform([OUTPUT_NUM_VOCAB, 
                                                   DECODER_EMBEDDING_DIM]))
decoder_input_embedded = tf.nn.embedding_lookup(decoder_embedding, 
                                                decoder_input_seq)

decoder_multi_cell = make_multi_cell(RNN_STATE_DIM, RNN_NUM_LAYERS)
 
output_layer_kernel_initializer = tf.truncated_normal_initializer(mean=0.0, stddev=0.1)
output_layer = Dense(
    OUTPUT_NUM_VOCAB,
    kernel_initializer = output_layer_kernel_initializer
)

In [15]:
with tf.variable_scope("decode"):
 
    training_helper = tf.contrib.seq2seq.TrainingHelper(
        inputs=decoder_input_embedded,
        sequence_length=decoder_seq_len,
        time_major=False
    )
 
    training_decoder = tf.contrib.seq2seq.BasicDecoder(
        decoder_multi_cell,
        training_helper,
        encoder_state,
        output_layer
    ) 
 
    training_decoder_output_seq, _, _ = tf.contrib.seq2seq.dynamic_decode(
        training_decoder, 
        impute_finished=True, 
        maximum_iterations=max_decoder_seq_len
    )

In [16]:
with tf.variable_scope("decode", reuse=True):
    start_tokens = tf.tile(
        tf.constant([output_symbol_to_int['<GO>']], 
                    dtype=tf.int32), 
        [BATCH_SIZE], 
        name='start_tokens')
 
    # Helper for the inference process.
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
        embedding=decoder_embedding,
        start_tokens=start_tokens,
        end_token=output_symbol_to_int['<EOS>']
    )
 
    # Basic decoder
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(
        decoder_multi_cell,
        inference_helper,
        encoder_state,
        output_layer
    )
 
    # Perform dynamic decoding using the decoder
    inference_decoder_output_seq, _, _ = tf.contrib.seq2seq.dynamic_decode(
        inference_decoder,
        impute_finished=True,
        maximum_iterations=40
    )

In [17]:
# rename the tensor for our convenience
training_logits = tf.identity(training_decoder_output_seq.rnn_output, name='logits')
inference_logits = tf.identity(inference_decoder_output_seq.sample_id, name='predictions')
# inference_logits = tf.expand_dims(inference_decoder_output_seq.sample_id, axis=0)
 
# Create the weights for sequence_loss
masks = tf.sequence_mask(
    decoder_seq_len, 
    max_decoder_seq_len, 
    dtype=tf.float32, 
    name='masks'
)
 
loss = tf.contrib.seq2seq.sequence_loss(
    training_logits,
    decoder_output_seq,
    masks
)

# Training summary for the current batch_loss
tf.summary.scalar('loss', loss)
summary_op = tf.summary.merge_all()


In [18]:
optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
 
gradients = optimizer.compute_gradients(loss)
capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var)
                        for grad, var in gradients if grad is not None]
train_op = optimizer.apply_gradients(capped_gradients)

In [19]:
def pad(xs, size, pad):
    return xs + [pad] * (size - len(xs))

In [27]:
input_seq = [
    [input_symbol_to_int.get(symbol, input_symbol_to_int['<UNK>']) 
        for symbol in line]  
    for line in input_sentences  
]
 
output_seq = [
    [output_symbol_to_int.get(symbol, output_symbol_to_int['<UNK>']) 
        for symbol in line] + [output_symbol_to_int['<EOS>']]  
    for line in output_sentences  
]

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()  
 

# step_per_checkpoint = 629

# test ----------------------------------------------------------------
step_per_checkpoint = 1
BATCH_SIZE = 5
NUM_EPOCS = 2
# ---------------------------------------------------------------------

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/seq2seq/' + current_time + '/train'
evaluation_log_dir = 'logs/seq2seq/' + current_time + '/evaluation'
bleu_test_log_dir = 'logs/seq2seq/' + current_time + '/test'
train_summary_writer = tf.summary.FileWriter(train_log_dir, graph=sess.graph)
evaluation_summary_writer = tf.summary.FileWriter(evaluation_log_dir, graph=sess.graph)
bleu_summary_writer = tf.summary.FileWriter(bleu_test_log_dir, graph=sess.graph)

for epoch in range(NUM_EPOCS + 1): 

    current_train_step = 0
    current_evaluate_step = 0
    # train
    print('Training Process ...')
    for batch_idx in range(len(train_input_sentences) // BATCH_SIZE): 
        
        input_batch, input_lengths, output_batch, output_lengths = [], [], [], []
        for sentence in train_input_sentences[batch_idx:batch_idx + BATCH_SIZE]:
            symbol_sent = [input_symbol_to_int[symbol] for symbol in sentence]
            padded_symbol_sent = pad(symbol_sent, MAX_CHAR_PER_LINE, input_symbol_to_int['<PAD>'])
            input_batch.append(padded_symbol_sent)
            input_lengths.append(len(sentence))
        for sentence in train_output_sentences[batch_idx:batch_idx + BATCH_SIZE]:
            symbol_sent = [output_symbol_to_int[symbol] for symbol in sentence]
            padded_symbol_sent = pad(symbol_sent, MAX_CHAR_PER_LINE, output_symbol_to_int['<PAD>'])
            output_batch.append(padded_symbol_sent)
            output_lengths.append(len(sentence))
        
        
        current_train_step += 1
        
        _, train_loss, train_summary = sess.run( 
            [train_op, loss, summary_op],
            feed_dict={
                encoder_input_seq: input_batch,
                encoder_seq_len: input_lengths,
                decoder_output_seq: output_batch,
                decoder_seq_len: output_lengths
            }
        )
        
        if current_train_step % step_per_checkpoint == 0:
            print('Epcoh {}. Batch {}/{}. Loss {}'.format(epoch, current_train_step, len(train_input_sentences) // BATCH_SIZE, train_loss))
    
    train_summary_writer.add_summary(train_summary, epoch)
        
    # evaluate 
    print('\nEvaluation Process ...')
    
    for batch_idx in range(len(evaluate_input_sentences) // BATCH_SIZE): 
        
        input_batch, input_lengths, output_batch, output_lengths = [], [], [], []
        for sentence in evaluate_input_sentences[batch_idx:batch_idx + BATCH_SIZE]:
            symbol_sent = [input_symbol_to_int[symbol] for symbol in sentence]
            padded_symbol_sent = pad(symbol_sent, MAX_CHAR_PER_LINE, input_symbol_to_int['<PAD>'])
            input_batch.append(padded_symbol_sent)
            input_lengths.append(len(sentence))
        for sentence in evaluate_output_sentences[batch_idx:batch_idx + BATCH_SIZE]:
            symbol_sent = [output_symbol_to_int[symbol] for symbol in sentence]
            padded_symbol_sent = pad(symbol_sent, MAX_CHAR_PER_LINE, output_symbol_to_int['<PAD>'])
            output_batch.append(padded_symbol_sent)
            output_lengths.append(len(sentence))
        
        current_evaluate_step += 1
        
        evaluation_loss, evaluate_summary, logits = sess.run( 
            [loss, summary_op, inference_logits],
            feed_dict={
                encoder_input_seq: input_batch,
                encoder_seq_len: input_lengths,
                decoder_output_seq: output_batch,
                decoder_seq_len: output_lengths
            }
        )
        
#         print(logits.shape)
            
        
        if current_evaluate_step % step_per_checkpoint == 0:
            print('Epcoh {}. Batch {}/{}. Loss {}'.format(epoch, current_evaluate_step, len(evaluate_input_sentences) // BATCH_SIZE, evaluation_loss))
    
    evaluation_summary_writer.add_summary(evaluate_summary, epoch)
    
    # test 
    print('\nTest Process...')

    bleu_epoch = 0
    bleu_val = 0
    for batch_idx in range(len(test_input_sentences) // BATCH_SIZE): 
        
        input_batch, input_lengths, output_voc_seq, output_voc_lengths = [], [], [], []
        for i, sentence in enumerate(test_input_sentences[batch_idx:batch_idx + BATCH_SIZE]):
            symbol_sent = [input_symbol_to_int[symbol] for symbol in sentence]
            padded_symbol_sent = pad(symbol_sent, MAX_CHAR_PER_LINE, input_symbol_to_int['<PAD>'])
            input_batch.append(padded_symbol_sent)
            input_lengths.append(len(sentence))


        true_char_seq = []
        test_ls = []
        for sentence in test_output_sentences[batch_idx:batch_idx + BATCH_SIZE]:
            test_ls.append(sentence)
            char_ls = []
            for char in sentence:
                char_ls.append(char)
            true_char_seq.append(char_ls)
        print(true_char_seq)
        print(test_ls)
#         break

        
        predict_seq = sess.run( 
            inference_logits,
            feed_dict={
                encoder_input_seq: input_batch,
                encoder_seq_len: input_lengths,

            }
        )
        
    
        predict_list = np.ndarray.tolist(predict_seq)
        print(predict_list)
    
        predict_char_seq = []
        for i in range(len(predict_list)):
            temp = []
            for idx in predict_list[i]:
                temp.append(output_int_to_symbol[idx])
            predict_char_seq.append(temp)
        
        print(predict_char_seq)
        
        sm_fn = SmoothingFunction()
        bleu_val += bleu_score.corpus_bleu(true_char_seq[:40], predict_char_seq, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=sm_fn.method1)
    
    bleu_epoch = bleu_val
    bleu_summary = tf.summary.scalar('bleu', bleu_epoch)
    bleu_summary_writer.add_summary(sess.run(bleu_summary), epoch)
        
# #     saver.save(sess, 'model.ckpt')   
sess.close()

Training Process ...
Epcoh 0. Batch 1/2. Loss 3.47695255279541
Epcoh 0. Batch 2/2. Loss 2.9931163787841797

Evaluation Process ...
Epcoh 0. Batch 1/2. Loss 2.9721853733062744
Epcoh 0. Batch 2/2. Loss 2.9771804809570312

Test Process...
[['f', 'u', 'c', 'k', ' ', 'i', 't', ' ', 'h', 'o', 'l', 'd', ' ', 't', 'i', 'g', 'h', 't', ' '], ['n', 'o', ' '], ['s', 'o', ' ', 't', 'e', 'l', 'l', ' ', 'm', 'e', ' ', 'm', 'o', 'r', 'e', ' ', 'a', 'b', 'o', 'u'], ['p', 'r', 'e', 't', 't', 'y', ' ', 'm', 'u', 'c', 'h', ' '], ['i', ' ', 'f', 'e', 'l', 'l', ' ', 'i', 'n', ' ', 'l', 'o', 'v', 'e', ' ', 'm', 'i', 'l', 'e', 's']]
['fuck it hold tight ', 'no ', 'so tell me more abou', 'pretty much ', 'i fell in love miles']
[[29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27], [29, 29, 29, 29, 29, 27, 29, 27, 27, 27, 27, 29, 29, 29, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 

InvalidArgumentError: You must feed a value for placeholder tensor 'decoder_output_seq' with dtype int32 and shape [?,?]
	 [[node decoder_output_seq (defined at <ipython-input-10-26c7cc45c629>:18) ]]

Original stack trace for 'decoder_output_seq':
  File "D:\miniconda3\envs\seq2seq_tf1\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\miniconda3\envs\seq2seq_tf1\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\miniconda3\envs\seq2seq_tf1\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\miniconda3\envs\seq2seq_tf1\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "D:\miniconda3\envs\seq2seq_tf1\lib\site-packages\ipykernel\kernelapp.py", line 619, in start
    self.io_loop.start()
  File "D:\miniconda3\envs\seq2seq_tf1\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
    self.asyncio_loop.run_forever()

In [24]:
%load_ext tensorboard
%tensorboard --logdir logs/seq2seq

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 16144), started 0:01:16 ago. (Use '!kill 16144' to kill it.)

In [23]:
sess = tf.InteractiveSession()    
saver.restore(sess, 'model.ckpt')

example_input_sent = "do you want to play games"
example_input_symb = [input_symbol_to_int[symbol] for symbol in example_input_sent]
example_input_batch = [pad(example_input_symb, MAX_CHAR_PER_LINE, input_symbol_to_int['<PAD>'])] * BATCH_SIZE
example_input_lengths = [len(example_input_sent)] * BATCH_SIZE

output_ints = sess.run(inference_logits, feed_dict={
    encoder_input_seq: example_input_batch,
    encoder_seq_len: example_input_lengths,
    decoder_seq_len: example_input_lengths
})[0]

output_str = ''.join([output_int_to_symbol[i] for i in output_ints])
print(output_str)

Instructions for updating:
Use standard file APIs to check for files with this prefix.


ValueError: The passed save_path is not a valid checkpoint: model.ckpt

In [ ]:
a = [1,2,3]
a[:6]